In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs

# !git config --global user.email "juliarochajunqueira@gmail.com"
# !git config --global user.name "Júlia"

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import BertForQuestionAnswering, TrainingArguments, Trainer
from torch.utils.data import DataLoader
from transformers import get_scheduler
from torch.optim import AdamW
import torch
from tqdm.auto import tqdm
from transformers import DefaultDataCollator
import numpy as np
from datasets import load_metric
from tqdm.auto import tqdm
import collections

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 66.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 69.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
 

In [ ]:
raw_datasets = load_dataset("squad_v1_pt")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset squad_v1_pt downloaded and prepared to /root/.cache/huggingface/datasets/squad_v1_pt/default/1.1.0/65162e0fbe44f19a4d2ad9f5f507d2e965e74249fc3239dc78b4e3bd93bab7c4. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print("Context: ", raw_datasets["train"][0]["context"])
print("Question: ", raw_datasets["train"][0]["question"])
print("Answer: ", raw_datasets["train"][0]["answers"])

Context:  Arquitetonicamente, a escola tem um caráter católico. No topo da cúpula de ouro do edifício principal é uma estátua de ouro da Virgem Maria. Imediatamente em frente ao edifício principal e de frente para ele, é uma estátua de cobre de Cristo com os braços erguidos com a lenda &quot;Venite Ad Me Omnes&quot;. Ao lado do edifício principal é a Basílica do Sagrado Coração. Imediatamente atrás da basílica é a Gruta, um lugar mariano de oração e reflexão. É uma réplica da gruta em Lourdes, na França, onde a Virgem Maria supostamente apareceu a Santa Bernadette Soubirous em 1858. No final da unidade principal (e em uma linha direta que liga através de 3 estátuas e da Cúpula de Ouro), é um estátua de pedra simples e moderna de Maria.
Question:  A quem a Virgem Maria supostamente apareceu em 1858 em Lourdes, na França?
Answer:  {'text': ['Saint Bernadette Soubirous'], 'answer_start': [0]}


In [ ]:
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
# small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [ ]:
max_length = 384
stride = 128
## preprocessamento do treinamento
def preprocess_training_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [ ]:
tokenized_train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)
len(raw_datasets["train"]), len(tokenized_train_dataset)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

(87599, 89800)

In [ ]:
##preprocessamento da validacao
def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

In [ ]:
tokenized_validation_dataset = raw_datasets["validation"].map(
    preprocess_validation_examples,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names,
)
len(raw_datasets["validation"]), len(tokenized_validation_dataset)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

(10570, 10956)

In [ ]:
small_train_dataset = tokenized_train_dataset.shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_validation_dataset.shuffle(seed=42).select(range(1000))

In [ ]:
print("Context: ", small_train_dataset[0])

Context:  {'input_ids': [101, 15807, 1085, 260, 1101, 111, 20633, 22286, 132, 9164, 649, 111, 20633, 22286, 132, 3364, 202, 1181, 125, 6444, 136, 102, 1494, 123, 495, 171, 2778, 6606, 161, 117, 5022, 11181, 271, 111, 20633, 22286, 132, 125, 549, 111, 20633, 22286, 132, 1085, 2134, 3198, 474, 122, 13961, 308, 119, 1645, 1061, 2365, 3015, 9638, 22281, 122, 117, 202, 1250, 117, 2365, 2955, 17796, 12535, 14219, 222, 1695, 2980, 171, 179, 11665, 15200, 271, 111, 20633, 22286, 132, 7769, 111, 20633, 22286, 132, 119, 231, 1161, 15758, 179, 259, 7769, 122, 259, 9164, 308, 3390, 22281, 6867, 173, 2276, 13019, 298, 8618, 117, 6752, 1491, 2221, 12597, 5533, 366, 2221, 271, 2276, 125, 7769, 119, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
tokenized_train_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 89800
})

In [ ]:
train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=16)
eval_dataloader = DataLoader(small_eval_dataset, batch_size=16)

#tamanho batch? conferir com o codigo do pierre

In [ ]:
trained_checkpoint = "neuralmind/bert-base-portuguese-cased"
model = BertForQuestionAnswering.from_pretrained(trained_checkpoint)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForQuestionAnswering: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-5)

In [ ]:
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
squad_v2 = False
model_checkpoint = "neuralmind/bert-base-portuguese-cased"
batch_size = 16
data_collator = DefaultDataCollator()

training_args = TrainingArguments(
    output_dir= "batata",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    fp16 = True
)

trainer = Trainer(
    model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)


trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.607300,No log
2,1.277400,No log
3,1.006200,No log


TrainOutput(global_step=16839, training_loss=1.3736035675297116, metrics={'train_runtime': 7666.3083, 'train_samples_per_second': 35.141, 'train_steps_per_second': 2.196, 'total_flos': 5.27950096985088e+16, 'train_loss': 1.3736035675297116, 'epoch': 3.0})

In [ ]:
trainer.save_model("batata")

In [ ]:
from google.colab import files
import shutil
!cp -av batata /content/drive/MyDrive/modelito


cp: cannot create directory '/content/drive/MyDrive/modelito': No such file or directory


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pad_on_right = tokenizer.padding_side == "right"
doc_stride = 128
max_length = 384

In [ ]:
for batch in trainer.get_eval_dataloader():
    break
batch = {k: v.to(trainer.args.device) for k, v in batch.items()}
with torch.no_grad():
    output = trainer.model(**batch)
output.keys()


odict_keys(['start_logits', 'end_logits'])

In [ ]:
output.start_logits.shape, output.end_logits.shape

(torch.Size([16, 384]), torch.Size([16, 384]))

In [ ]:
n_best_size = 20

start_logits = output.start_logits[0].cpu().numpy()
end_logits = output.end_logits[0].cpu().numpy()
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if start_index <= end_index: # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "" # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )



In [ ]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples


In [ ]:
validation_features = raw_datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=raw_datasets["validation"].column_names
)

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [ ]:
print(validation_features)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 10956
})


In [ ]:
raw_predictions = trainer.predict(validation_features)

In [ ]:
print(raw_predictions)

PredictionOutput(predictions=(array([[-6.7695312, -8.       , -8.328125 , ..., -8.453125 , -8.4765625,
        -8.46875  ],
       [-6.6757812, -7.578125 , -8.265625 , ..., -8.3984375, -8.4140625,
        -8.421875 ],
       [-6.7226562, -8.171875 , -8.4453125, ..., -8.5390625, -8.390625 ,
        -8.5390625],
       ...,
       [-6.4453125, -8.15625  , -8.3046875, ..., -8.5078125, -8.4375   ,
        -8.53125  ],
       [-6.2226562, -7.9453125, -8.5      , ..., -8.4296875, -8.3828125,
        -8.3984375],
       [-6.4179688, -8.09375  , -8.4296875, ..., -8.4453125, -8.390625 ,
        -8.4765625]], dtype=float32), array([[-6.9023438, -8.484375 , -8.3359375, ..., -8.6640625, -8.65625  ,
        -8.6484375],
       [-6.9453125, -8.546875 , -8.203125 , ..., -8.7109375, -8.6875   ,
        -8.6875   ],
       [-6.8671875, -8.3671875, -8.265625 , ..., -8.5703125, -8.65625  ,
        -8.5703125],
       ...,
       [-6.671875 , -8.5      , -8.375    , ..., -8.625    , -8.671875 ,
        -8

In [ ]:
validation_features.set_format(type=validation_features.format["type"], columns=list(validation_features.features.keys()))

In [ ]:
print(validation_features)

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_id'],
    num_rows: 10956
})


In [ ]:
def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 30):
    all_start_logits, all_end_logits = raw_predictions
    # Build a map example to its corresponding features.

    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.

            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = best_answer["text"] if best_answer["score"] > min_null_score else ""
            predictions[example["id"]] = answer

    return predictions



In [ ]:
final_predictions = postprocess_qa_predictions(raw_datasets["validation"], validation_features, raw_predictions.predictions)

metric = load_metric("squad_v2" if squad_v2 else "squad")

print(final_predictions)


Post-processing 10570 example predictions split into 10956 features.


  0%|          | 0/10570 [00:00<?, ?it/s]

<ipython-input-33-6db034637fec>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("squad_v2" if squad_v2 else "squad")


OrderedDict([('56be4db0acb8001400a502ec', 'Denver Broncos'), ('56be4db0acb8001400a502ed', 'Carolina Panthers'), ('56be4db0acb8001400a502ee', 'Super Bowl 50 foi um jogo de futebol'), ('56be4db0acb8001400a502ef', 'Denver Broncos'), ('56be4db0acb8001400a502f0', 'ouro'), ('56be8e613aeaaa14008c90d1', 'Super Bowl 50 foi um jogo de futebol americano'), ('56be8e613aeaaa14008c90d2', '7 de fevereiro de 2016'), ('56be8e613aeaaa14008c90d3', 'Super Bowl 50 foi um jogo de futebol'), ('56bea9923aeaaa14008c91b9', 'Super Bowl 50 foi um jogo de futebol americano'), ('56bea9923aeaaa14008c91ba', 'Super Bowl 50 foi um jogo'), ('56bea9923aeaaa14008c91bb', '7 de fevereiro de 2016'), ('56beace93aeaaa14008c91df', 'Denver Broncos'), ('56beace93aeaaa14008c91e0', 'Super Bowl 50 foi um jogo de futebol'), ('56beace93aeaaa14008c91e1', 'Santa Clara'), ('56beace93aeaaa14008c91e2', 'Super Bowl L'), ('56beace93aeaaa14008c91e3', '2015'), ('56bf10f43aeaaa14008c94fd', '2016'), ('56bf10f43aeaaa14008c94fe', 'Santa Clara'), (

In [ ]:
print(raw_predictions.predictions)

(array([[-6.7695312, -8.       , -8.328125 , ..., -8.453125 , -8.4765625,
        -8.46875  ],
       [-6.6757812, -7.578125 , -8.265625 , ..., -8.3984375, -8.4140625,
        -8.421875 ],
       [-6.7226562, -8.171875 , -8.4453125, ..., -8.5390625, -8.390625 ,
        -8.5390625],
       ...,
       [-6.4453125, -8.15625  , -8.3046875, ..., -8.5078125, -8.4375   ,
        -8.53125  ],
       [-6.2226562, -7.9453125, -8.5      , ..., -8.4296875, -8.3828125,
        -8.3984375],
       [-6.4179688, -8.09375  , -8.4296875, ..., -8.4453125, -8.390625 ,
        -8.4765625]], dtype=float32), array([[-6.9023438, -8.484375 , -8.3359375, ..., -8.6640625, -8.65625  ,
        -8.6484375],
       [-6.9453125, -8.546875 , -8.203125 , ..., -8.7109375, -8.6875   ,
        -8.6875   ],
       [-6.8671875, -8.3671875, -8.265625 , ..., -8.5703125, -8.65625  ,
        -8.5703125],
       ...,
       [-6.671875 , -8.5      , -8.375    , ..., -8.625    , -8.671875 ,
        -8.6171875],
       [-6.3789062

In [ ]:
print(final_predictions.items())

odict_items([('56be4db0acb8001400a502ec', 'Denver Broncos'), ('56be4db0acb8001400a502ed', 'Carolina Panthers'), ('56be4db0acb8001400a502ee', 'Super Bowl 50 foi um jogo de futebol'), ('56be4db0acb8001400a502ef', 'Denver Broncos'), ('56be4db0acb8001400a502f0', 'ouro'), ('56be8e613aeaaa14008c90d1', 'Super Bowl 50 foi um jogo de futebol americano'), ('56be8e613aeaaa14008c90d2', '7 de fevereiro de 2016'), ('56be8e613aeaaa14008c90d3', 'Super Bowl 50 foi um jogo de futebol'), ('56bea9923aeaaa14008c91b9', 'Super Bowl 50 foi um jogo de futebol americano'), ('56bea9923aeaaa14008c91ba', 'Super Bowl 50 foi um jogo'), ('56bea9923aeaaa14008c91bb', '7 de fevereiro de 2016'), ('56beace93aeaaa14008c91df', 'Denver Broncos'), ('56beace93aeaaa14008c91e0', 'Super Bowl 50 foi um jogo de futebol'), ('56beace93aeaaa14008c91e1', 'Santa Clara'), ('56beace93aeaaa14008c91e2', 'Super Bowl L'), ('56beace93aeaaa14008c91e3', '2015'), ('56bf10f43aeaaa14008c94fd', '2016'), ('56bf10f43aeaaa14008c94fe', 'Santa Clara'), (

In [ ]:
if squad_v2:
    formatted_predictions = [{"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in final_predictions.items()]
else:
    formatted_predictions = [{"id": k, "prediction_text": v} for k, v in final_predictions.items()]


print(formatted_predictions[0])
print(raw_datasets["validation"]["answers"][0])

references = [{"id": ex["id"], "answers": ex["answers"]} for ex in raw_datasets["validation"]]
metric.compute(predictions=formatted_predictions, references=references)



{'id': '56be4db0acb8001400a502ec', 'prediction_text': 'Denver Broncos'}
{'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos'], 'answer_start': [182, 182, 182]}


{'exact_match': 43.292336802270576, 'f1': 56.4826850136212}